In [4]:
import numpy as np 
import pandas as pd

index = pd.date_range("1/1/2000", periods=8)
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])
df = pd.DataFrame(np.random.randn(8, 3), index=index, columns=["A", "B", "C"])

In [ ]:
# Function application
# https://pandas.pydata.org/docs/user_guide/basics.html#function-application

# Tablewise Function Application: pipe()
# Row or Column-wise Function Application: apply()
# Aggregation API: agg() and transform()
# Applying Elementwise Functions: applymap()

In [ ]:
# Tablewise function application
# 주고 받는 단위가 Table이란 의미

In [5]:
def extract_city_name(df):
    """
    df를 argument로 하고 있다.
    Chicago, IL -> Chicago for city_name column
    """
    df["city_name"] = df["city_and_code"].str.split(",").str.get(0)
    return df


def add_country_name(df, country_name=None):
    """
    Chicago -> Chicago-US for city_name column
    """
    # col = "city_name"
    df["city_and_country"] = df["city_name"] + "-" + country_name
    return df

In [6]:
df_p = pd.DataFrame({"city_and_code": ["Chicago, IL"]})
df_p

,city_and_code
0,"Chicago, IL"


In [7]:
add_country_name(extract_city_name(df_p), country_name="US")

,city_and_code,city_name,city_and_country
0,"Chicago, IL",Chicago,Chicago-US


In [ ]:
# pipe
df_p.pipe(extract_city_name).pipe(add_country_name, country_name="US")


* <span style='font-size:13px;'>기존에는 함수에 argument로 DataFrmae을 전달하여야 하나, pipe를 쓴다는 것은, 함수가 아닌 `DataFrame에` Pipe를 통하여 함수를 직접 적용한다.
* <span style='font-size:13px;'>위의 예에서 볼 수 있듯이 code의 가독성이 더 좋다.

In [ ]:
import statsmodels.formula.api as sm

bb = pd.read_csv("baseball.csv", index_col="id")

(
    bb.query("h > 0")
    .assign(ln_h=lambda df: np.log(df.h))
    .pipe((sm.ols, "data"), "hr ~ ln_h + year + g + C(lg)")
    .fit()
    .summary()
)

In [ ]:
# Row or column-wise function application
# 이는 apply() Method에 의해 구현된다.    🔰🔰
# apply method를 적용하면, 자동으로 df, series로 부터 row, column을 받아 오는 방식이다. 적용하는 func의 처리를 거쳐
# return값으로 column 이나 row의 index를 index로 갖는 Series나 DataFrame을 반환한다.

In [ ]:
tsdf = pd.DataFrame(
    np.random.randn(10, 3),
    columns=['A', 'B', 'C'],
    index=pd.date_range('1/1/2017', periods=10)
)

tsdf.apply(lambda x: x.idxmax())
#  여기서 x는 column을 하나씩 불러 들인 것이다.따라서 return갑은 tsdf의 columns=[...]를 index로하는 Series가 된다

In [ ]:
df

In [ ]:
def subtract_and_divide(x, sub, divide=1):
    return(x-sub) / divide
df.apply(subtract_and_divide, args=(5,), divide=3)

In [ ]:
tsdf.iloc[3:7] = np.nan
tsdf

In [ ]:
tsdf.apply(pd.Series.interpolate)

In [ ]:
s = pd.Series([np.nan, "single_one", np.nan,
               "fill_two_more", np.nan, np.nan, np.nan,
               4.71, np.nan])
s

In [ ]:
s.interpolate(method='pad', limit=2)
# NaN을 채우는데 연속적인 2개의 NaN밖에 못채운다. 

In [ ]:
# Aggregation API
# Aggregating with multiple functions

In [ ]:
df = pd.DataFrame([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [np.nan, np.nan, np.nan]],
                  columns=['A', 'B', 'C'])
df

In [ ]:
df.agg(['sum', 'min'])

In [ ]:
df.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']})

In [ ]:
df.agg(x=('A', max), y=('B', 'min'), z=('C', np.mean))
# 여기서 x, y, z는 함수를 정의한다.   🔰💢


In [ ]:
# lambda
tsdf["A"].agg(["sum", lambda x: x.mean()])


In [ ]:
# 정의 함수
def mymean(x):
    return x.mean()

tsdf["A"].agg(["sum", mymean])


In [ ]:
# Aggregating with a dict
# 어떤 column(row)에 어떤 함수를 적용할 것인가를 Dictionary에서 정의해서 준다.
tsdf.agg({'A': 'mean', 'B':'sum'})